In [0]:
import pyspark.sql.functions as f
from pyspark.sql.types import TimestampType
from pyspark.sql.types import StringType
from pyspark.sql.functions import *

In [0]:
#Fact_payment table
payments = spark.table("staging_payments")
display(payments.limit(5))

payment_id,payment_date,amount,rider_id
1574726,2021-02-01,9.0,61831
1574727,2021-03-01,9.0,61831
1574728,2021-04-01,9.0,61831
1574729,2021-05-01,9.0,61831
1574730,2021-06-01,9.0,61831


In [0]:
#Write data to fact_payment
spark.sql('DROP TABLE IF EXISTS fact_payment')
payments.dropDuplicates(['payment_id']).write.format('delta').mode('overwrite').saveAsTable('fact_payment')

In [0]:
fact_payment= spark.table('fact_payment')
display(fact_payment.limit(5))
     

payment_id,payment_date,amount,rider_id
148,2017-09-01,6.86,1007
463,2021-09-01,22.35,1018
471,2021-10-01,12.52,1019
496,2021-11-01,9.0,1020
833,2021-04-01,9.0,1035


In [0]:
#Dim_rider Table

riders = spark.table('staging_riders')
display(riders.limit(5))

rider_id,first_name,last_name,address,birthday,account_start_date,account_end_date,is_member
1000,Diana,Clark,1200 Alyssa Squares,1989-02-13,2019-04-23,null,true
1001,Jennifer,Smith,397 Diana Ferry,1976-08-10,2019-11-01,2020-09-01,true
1002,Karen,Smith,644 Brittany Row Apt. 097,1998-08-10,2022-02-04,null,true
1003,Bryan,Roberts,996 Dickerson Turnpike,1999-03-29,2019-08-26,null,false
1004,Jesse,Middleton,7009 Nathan Expressway,1969-04-11,2019-09-14,null,true


In [0]:

#Write data to dim_rider
spark.sql('DROP TABLE IF EXISTS dim_rider')
riders.dropDuplicates(['rider_id']).write.format('delta').mode('overwrite').saveAsTable('dim_rider')

In [0]:
dim_rider = spark.table('dim_rider')
display(dim_rider.limit(5))

rider_id,first_name,last_name,address,birthday,account_start_date,account_end_date,is_member
1236,Julia,Mack,64964 Michelle Wall Suite 298,1990-05-20,2020-04-24,null,true
1661,David,Jones,37516 Crawford Heights Suite 735,1996-10-16,2018-11-12,null,false
1810,John,Harvey,6701 Watkins Passage,1977-02-27,2019-02-15,null,true
1914,Shannon,Preston,88501 Zhang Valley Suite 076,1990-08-13,2018-08-25,null,true
1998,Veronica,Long,91058 Carson Circle,1990-03-12,2021-05-28,2022-01-01,true


In [0]:
#Dim_Station Table

stations = spark.table("default.staging_stations")

display(stations.limit(5))

station_id,name,latitude,longitude
525,Glenwood Ave & Touhy Ave,42.012701,-87.66605799999999
KA1503000012,Clark St & Lake St,41.88579466666667,-87.63110066666668
637,Wood St & Chicago Ave,41.895634,-87.672069
13216,State St & 33rd St,41.8347335,-87.6258275
18003,Fairbanks St & Superior St,41.89580766666667,-87.62025316666669


In [0]:
#Write data to dim_station
spark.sql("drop table if exists default.dim_station")
stations.dropDuplicates(["station_id"]).write.format("delta").mode("overwrite").saveAsTable("default.dim_station")

In [0]:
#Fact_Trip Table
trips = spark.table('staging_trips')
display(trips.limit(5))
     

trip_id,rideable_type,start_at,ended_at,start_station_id,end_station_id,rider_id
7E1E50AC37E2DAD3,classic_bike,2021-08-14T14:01:36Z,2021-08-14T14:34:49Z,TA1309000007,13089,2644
ADFF32195521E952,classic_bike,2021-08-29T16:16:36Z,2021-08-29T16:24:43Z,13288,TA1308000031,37747
7C59843DB8D13CC7,electric_bike,2021-08-27T11:06:34Z,2021-08-27T11:12:52Z,TA1307000062,TA1305000020,63224
5B788004F8A5204C,classic_bike,2021-08-27T07:35:33Z,2021-08-27T07:59:35Z,13353,13242,45050
078629DD14B634AE,classic_bike,2021-08-08T15:00:30Z,2021-08-08T15:22:57Z,13353,13242,33762


In [0]:
#Write data to fact_trip
fact_trips = trips.alias("tbl1").join(riders.alias("tbl2"), col("tbl1.rider_id") == col("tbl2.rider_id"), "left") \
            .withColumn('duration', round((unix_timestamp("ended_at") - unix_timestamp('start_at'))/60)) \
            .withColumn('rider_age', round((unix_timestamp("account_start_date") - unix_timestamp('birthday'))/3600/24)) \
            .withColumnRenamed('start_at', 'start_time_id') \
            .withColumnRenamed('ended_at', 'end_time_id') \
            .select("trip_id", col("tbl1.rider_id"), "rideable_type", "start_station_id", "end_station_id", "start_time_id", "end_time_id", "duration", "rider_age")
            
display(fact_trips.limit(5))

trip_id,rider_id,rideable_type,start_station_id,end_station_id,start_time_id,end_time_id,duration,rider_age
7E1E50AC37E2DAD3,2644,classic_bike,TA1309000007,13089,2021-08-14T14:01:36Z,2021-08-14T14:34:49Z,33.0,16081.0
ADFF32195521E952,37747,classic_bike,13288,TA1308000031,2021-08-29T16:16:36Z,2021-08-29T16:24:43Z,8.0,7306.0
7C59843DB8D13CC7,63224,electric_bike,TA1307000062,TA1305000020,2021-08-27T11:06:34Z,2021-08-27T11:12:52Z,6.0,12635.0
5B788004F8A5204C,45050,classic_bike,13353,13242,2021-08-27T07:35:33Z,2021-08-27T07:59:35Z,24.0,10321.0
078629DD14B634AE,33762,classic_bike,13353,13242,2021-08-08T15:00:30Z,2021-08-08T15:22:57Z,22.0,16062.0


In [0]:
#Dim_date Table

from pyspark.sql import functions as f
from pyspark.sql.types import StringType

# Get min and max date from trips
max_date = trips.selectExpr('MAX(start_at) AS start_at').first().asDict()['start_at']
min_date = trips.selectExpr('MIN(start_at) AS start_at').first().asDict()['start_at']
expression = f"sequence(to_date('{min_date}'), to_date('{max_date}'), interval 1 day)"
dim_date = spark.createDataFrame([(1,)], ["time_id"])

dim_date = dim_date.withColumn("dateinit", f.explode(f.expr(expression)))
dim_date = dim_date.withColumn("date", f.to_timestamp(dim_date.dateinit, "yyyy-MM-dd"))

dim_date = dim_date \
            .withColumn("day", f.dayofmonth(dim_date.date)) \
            .withColumn("month", f.month(dim_date.date)) \
            .withColumn("quarter", f.quarter(dim_date.date)) \
            .withColumn("year", f.year(dim_date.date)) \
            .withColumn("day_of_week", f.dayofweek(dim_date.date)) \
            .withColumn("week_of_year", f.weekofyear(dim_date.date)) \
            .withColumn("time_id", dim_date.date.cast(StringType())) \
            .drop(f.col("dateinit"))

In [0]:
#checking metrics
display(dim_date.selectExpr('MIN(date)').show())
display(dim_date.selectExpr('MAX(date)').show())

+-------------------+
|          min(date)|
+-------------------+
|2021-02-01 00:00:00|
+-------------------+

+-------------------+
|          max(date)|
+-------------------+
|2022-01-31 00:00:00|
+-------------------+



In [0]:
# Write data to dim_date
spark.sql("drop table if exists default.dim_date")
dim_date.write.format("delta").mode("overwrite").saveAsTable("default.dim_date")